### CV 딥러닝

**4월 29일**

In [1]:
import matplotlib.pyplot as plt
import cv2 as cv
import numpy as np
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten
from keras.models import Sequential
from keras.utils import to_categorical, pad_sequences
from keras.layers import BatchNormalization, Dropout

from glob import glob



2023-04-29 10:24:48.763521: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
base_dir = './kagglecatsanddogs_5340/PetImages'
file_list_dog = glob(base_dir+'/Dog/*.jpg')[:5000]
file_list_cat = glob(base_dir+'/Cat/*.jpg')[:5000]

In [3]:
file_list = np.concatenate([file_list_dog, file_list_cat], axis=0)
# t = plt.imread(file_list[0])
# im_rgb = cv.cvtColor(t, cv.COLOR_BGR2RGB)
# print(im_rgb.shape)
# plt.imshow(im_rgb)

X = np.zeros((len(file_list),150,150,3))
# Y = np.concatenate([np.zeros(500,1), np.ones(500,1)], axis=0)
y = []
# resize 시 except 발생하는 file list 정리하기
# 
# invalid_index = []
# valid_index = []

# for i, file in enumerate(file_list):
#     try:
#         dt = plt.imread(file)
#         dt = cv.resize(file, (150,150))
#         valid_index.append(i)
#     except:
#         invalid_index.append(i)

# print(invalid_index)

# file_list = file_list[valid_index]

# 방법 2: data가 없는 파일 제거함.
#         --> 제거 후에 나머지 이미지 파일은 channel 통일시킴
for i, file in enumerate(file_list):
    t = plt.imread(file)
    f = cv.cvtColor(t, cv.COLOR_BGR2RGB)
    f = cv.resize(f, (150,150))
    if 'Cat' in file:
        y.append(0)
    else:
        y.append(1)

    # if f.shape[-1] == 4:
    #     f = f[:,:,:3]
    # elif f.shape[-1] > 4:
    #     print(i)

    X[i] = f

y = np.array(y)
# print(y)

/home/nextchoice/anaconda3/envs/deep/lib/python3.10/site-packages/PIL/TiffImagePlugin.py:858: UserWarning: Truncated File Read
  warnings.warn(str(msg))


In [4]:
# modeling
model = Sequential([
    Conv2D(50, (3,3), activation='relu', input_shape=(150,150,3)),
    BatchNormalization(),
    MaxPooling2D((5,5)),
    Conv2D(50, (3,3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D((7,7)),
    Flatten(),
    Dense(50, activation='relu'),
    BatchNormalization(),
    Dense(1, activation='sigmoid')
])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 148, 148, 50)      1400      
                                                                 
 batch_normalization (BatchN  (None, 148, 148, 50)     200       
 ormalization)                                                   
                                                                 
 max_pooling2d (MaxPooling2D  (None, 29, 29, 50)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 27, 27, 50)        22550     
                                                                 
 batch_normalization_1 (Batc  (None, 27, 27, 50)       200       
 hNormalization)                                                 
                                                        

2023-04-29 10:25:10.077711: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-29 10:25:10.079688: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


In [5]:
opt = 'adam'
loss = 'binary_crossentropy'
metrics = ['accuracy']
model.compile(optimizer=opt,
              loss=loss,
              metrics=metrics)
# index = np.random.choice(np.arrange(file_list), file_list.size, replace=False)
index = np.random.choice(np.arange(len(X)), len(X), replace=False)
tr_ind = index[:int(index.size*0.8)]
te_idd = index[int(index.size*0.8):]

X_train = X[tr_ind]
y_train = y[tr_ind]

X_test = X[te_idd]
y_test = y[te_idd]

# model.fit(X_train, y_train, epochs=10, batch_size=1000, validation_data=(X_test, y_test))
model.fit(X_train, y_train, epochs=10, batch_size=1000)

Epoch 1/10


: 

: 

In [ ]:
# 모델2
# input_shape = X_train.shape[1:]
# print(input_shape)

# model = Sequential([
#     Conv2D(32, (3,3), activation='relu', input_shape=(150,150,3)),
#     MaxPooling2D((2,2)),
#     Conv2D(64, (3,3), activation='relu'),
#     Conv2D(128, (5,5), activation='relu'),
#     MaxPooling2D((5,5)),
#     Flatten(),
#     Dense(128, activation='relu'),
#     Dense(1, activation='sigmoid')
# ])

# opt = 'adam'
# loss = 'binary_crossentropy'
# metrics = ['accuracy']
# model.compile(optimizer=opt,
#               loss=loss,
#               metrics=metrics)

# model.fit(X_train, y_train, epochs=10, batch_size=128)

(150, 150, 3)


In [ ]:
# 테스트 정확도를 출력합니다.
print("\n Test Accuracy: %.4f" % (model.evaluate(X_test, y_test)[1]))